# YOLO Model training

## Importations and initialisations

### Librairies

In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
from utils import init_notebook
%aimport datasets, datasets.data_transform
import datasets

import torch
import fiftyone as fo
from fiftyone import ViewField as F
from ultralytics import YOLO
import numpy as np
import os


from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction

from pathlib import Path

HOME = Path(os.getcwd()).parents[0]
HOME


WindowsPath('c:/Users/Geomatique/Documents/map-symbols-detection-in-historical-maps/notebooks/Sutty_pipeline')

## Base Model importation 
Optional it is now called later directlry in the training kernel

In [3]:
from ultralytics import YOLO

detection_model = YOLO("yolov10x.pt")

## Data Importation

### Roboflow
Obsolete : Only if you need to import your datasets from Roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="VsgwOHYfQtqRLRCUpPv7")
project = rf.workspace("suttyworkspace").project("cassini_maps")
version = project.version(2)
dataset = version.download("yolov8")

### Model training

__If the training is not launching, check the paths set in your yaml file__

In [8]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

detection_model = YOLO("yolov10x.pt") # The base model is imported here
experiment = Experiment(
  api_key="<api-key>",    #your COMET API key
  project_name="<project-name>",    # your project name
  workspace="<username>"                   # your username
  #experiment_key = f"yolov10x_f52_{epochs}ep_imgsz1200_aug"
) 

#Hyperparameters
epochs = 30

imgsz = 640


results=detection_model.train(data=(HOME/"data/yolo_datasets/Yolass_aug/data.yaml").as_posix(), #Path to the data.yaml file
                    batch=8, #8,    # If you put -1, the batch size will be automatically calculated based on 60% of the GPU memory
                    epochs=epochs,  # epochs
                    imgsz=imgsz,  # Image size
                    plots=True,
                    pretrained=True,
                    project= (HOME/"runs/yolo").as_posix(), # Path to the project directory (where the logs and the model weigths will be saved)
                    name= f"yolov10x_{epochs}_ep__{imgsz}imgs_Yolass_aug_v2",
                    save_dir= "train",  
                    patience=2,   # Patience for Early Stopping
                    verbose=False,
                    cache=True, #Stores dataset images in RAM, providing the fastest access speed but at the cost of increased memory usage.
                    amp=True, # Mixed precision training
                    cls=1.0, #Weight of the classification loss (default 0.5) It can help when some classes are underrepresented but beware of overfitting
                    ) #Training of the model with the COCASS dataset under YOLO format

experiment.end()    #Do not forget to end the experiment to save the results in Comet 
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : spontaneous_pediment_8857
COMET INFO:     url                   : https://www.comet.com/charsutty/yolov8-ehess-cassini/c509fa61dde44667ba462de015331f65
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (6.87 MB)
COMET INFO:     installed packages       : 1
COMET INFO:     notebook                 : 1
COMET INFO:     source_code              : 1
COMET INFO: 
COMET WARNING: To get all data

New https://pypi.org/project/ultralytics/8.2.79 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.55  Python-3.12.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov10x.pt, data=c:/Users/Geomatique/Documents/map-symbols-detection-in-historical-maps/notebooks/Sutty_pipeline/data/yolo_datasets/Yolass_aug/data.yaml, epochs=30, time=None, patience=2, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=c:/Users/Geomatique/Documents/map-symbols-detection-in-historical-maps/notebooks/Sutty_pipeline/runs/yolo, name=yolov10x_30_ep__640imgs_Yolass_aug_v219, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False

train: Scanning C:\Users\Geomatique\Documents\map-symbols-detection-in-historical-maps\notebooks\Sutty_pipeline\data\yolo_datasets\Yolass_aug\train\labels.cache... 6383 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6383/6383 [00:00<?, ?it/s]
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000020BB9641620>
Traceback (most recent call last):
  File "c:\Users\Geomatique\Documents\map-symbols-detection-in-historical-maps\ehess_env\Lib\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "c:\Users\Geomatique\Documents\map-symbols-detection-in-historical-maps\ehess_env\Lib\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
train: Caching images (7.3GB RAM): 100%|██████

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\Geomatique\Documents\map-symbols-detection-in-historical-maps\notebooks\Sutty_pipeline\data\yolo_datasets\Yolass_aug\val\labels... 595 images, 0 backgrounds, 0 corrupt: 100%|██████████| 595/595 [00:00<00:00, 2937.87it/s]

...

                   all        595      15435      0.944       0.82       0.89      0.727
EarlyStopping: Training stopped early as no improvement observed in last 2 epochs. Best results observed at epoch 24, best model saved as best.pt.
To update EarlyStopping(patience=2) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



26 epochs completed in 2.132 hours.
Optimizer stripped from c:\Users\Geomatique\Documents\map-symbols-detection-in-historical-maps\notebooks\Sutty_pipeline\runs\yolo\yolov10x_30_ep__640imgs_Yolass_aug_v219\weights\last.pt, 64.2MB
Optimizer stripped from c:\Users\Geomatique\Documents\map-symbols-detection-in-historical-maps\notebooks\Sutty_pipeline\runs\yolo\yolov10x_30_ep__640imgs_Yolass_aug_v219\weights\best.pt, 64.2MB

Validating c:\Users\Geomatique\Documents\map-symbols-detection-in-historical-maps\notebooks\Sutty_pipeline\runs\yolo\yolov10x_30_ep__640imgs_Yolass_aug_v219\weights\best.pt...
Ultralytics YOLOv8.2.55  Python-3.12.4 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv10x summary (fused): 503 layers, 31,616,822 parameters, 0 gradients, 169.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:09<00:00,  3.98it/s]


                   all        595      15435       0.95      0.819      0.897      0.728
Speed: 0.1ms preprocess, 11.4ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to c:\Users\Geomatique\Documents\map-symbols-detection-in-historical-maps\notebooks\Sutty_pipeline\runs\yolo\yolov10x_30_ep__640imgs_Yolass_aug_v219


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : early_albatross_6820
COMET INFO:     url                   : https://www.comet.com/charsutty/yolov8-ehess-cassini/583a37e6c25b4e318b5cd613b596c0d2
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [53]               : (8.329999999999999e-05, 0.0004443982923976608)
COMET INFO:     lr/pg1 [53]               : (8.329999999999999e-05, 0.0004443982923976608)
COMET INFO:     lr/pg2 [53]               : (8.329999999999999e-05, 0.0004443982923976608)
COMET INFO:     metrics/mAP50(B) [54]     : (0.5199, 0.89941)
COMET INFO:     metrics/mAP50-95(B) [54]  : (0.31336, 0.7282208147228211)
COMET INFO:     metrics/precision(B) [54] : (0.69088, 0.9521

You can also inspect the logs with tensorboard

In [ ]:
!tensorboard --logdir runs/yolo/